In [1]:
import ee
import sys
import os
import re
import math
import itertools

sys.path.append(os.path.abspath('../'))

from modules.SpectralIndexes import *
from modules.Miscellaneous import *
from modules.Mosaic import *
from modules.SmaAndNdfi import * 
from modules import Map

ee.Initialize()

In [2]:
## Define version to be processed 
samples_version = '11'     ## version string
output_version  = '11'

## Set folder .multiply(1000).round(.multiply(1000).round())
output_asset = 'projects/mapbiomas-workspace/COLECAO_DEV/COLECAO9_DEV/CERRADO/C9-GENERAL-MAP-PROBABILITY/'

## Set regions ids
regions_list = list(range(1, 39))
years = list(range(1985, 2023))

## set mosaic date range
dateStart = '-01-01'
dateEnd = '-12-31'

## list files in the folder 
files = ee.data.listAssets({'parent': output_asset})
files = [asset['name'] for asset in files['assets']]


# Remove the prefix
files = [file.replace('projects/earthengine-legacy/assets/', '') for file in files]

# Assuming regions_list, years, output_asset, and output_version are defined
expected = [
    f"{output_asset}CERRADO_{region}_{year}_v{output_version}"
    for region, year in itertools.product(regions_list, years)
]

# Find missing entries
missing = [entry for entry in expected if entry not in files]

# Define class dictionary
class_dict = {
    "class": [3, 4, 11, 12, 15, 18, 25, 33],
    "name": ['Forest', 'Savanna', 'Wetland', 'Grassland', 'Pasture', 'Agriculture', 'Non-Vegetated', 'Water']
}

## training dir
training_dir = 'projects/mapbiomas-workspace/COLECAO_DEV/COLECAO9_DEV/CERRADO/training/'

# Biome layer
biomes = ee.Image('projects/mapbiomas-workspace/AUXILIAR/biomas-2019-raster')
cerrado = biomes.updateMask(biomes.eq(4))

## Import classification regions
regions_vec = ee.FeatureCollection('users/dh-conciani/collection7/classification_regions/vector_v2')

## Classification regions (imageCollection, one region per image)
regions_ic = 'users/dh-conciani/collection7/classification_regions/eachRegion_v2_10m/'

## Landsat collections
collectionId = 'LANDSAT/COMPOSITES/C02/T1_L2_32DAY'

## spectral bands selected
spectralBands = ['blue', 'red', 'green', 'nir', 'swir1', 'swir2']

## endemembers collection
endmembers = ENDMEMBERS['landsat-8']

## Time since last fire
fire_age = ee.Image('users/barbarasilvaIPAM/collection8/masks/fire_age_v2')
## add years
fire_age = fire_age.addBands(fire_age.select('classification_2022').rename('classification_2023'))\
    .addBands(fire_age.select('classification_2022').rename('classification_2024'))

# Extract the region using regex
#regions_list = list(set(re.sub(r".*CERRADO_([0-9]+)_.*", r"\1", item) for item in missing))
regions_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', 
                '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
                '30', '31', '32', '33', '34', '35', '36', '37', '38']

In [3]:
for i in regions_list:
    print('processing region: ' + i)

    ## Get the vector for the region [i]
    region_i_vec = regions_vec.filterMetadata('mapb', 'equals', int(i)).geometry()

    ## Get the raster for the region [i]
    region_i_ras = ee.Image(regions_ic +'reg_' + i)

    ## Compute additional bands
    geo_coordinates = ee.Image.pixelLonLat().clip(region_i_vec)
  
    ## Get latitude
    lat = geo_coordinates.select('latitude')\
        .add(5)\
        .multiply(-1)\
        .multiply(1000)\
        .toInt16()
        
    ## Get longitude
    lon_sin = geo_coordinates.select('longitude')\
        .multiply(math.pi)\
        .divide(180)\
        .sin()\
        .multiply(-1)\
        .multiply(10000)\
        .toInt16()\
        .rename('longitude_sin')
    
    ## Cosine
    lon_cos = geo_coordinates.select('longitude')\
        .multiply(math.pi)\
        .divide(180)\
        .cos()\
        .multiply(-1)\
        .multiply(10000)\
        .toInt16()\
        .rename('longitude_cos')
    
    ## Get heigth above nearest drainage
    hand = ee.ImageCollection("users/gena/global-hand/hand-100")\
        .mosaic()\
        .toInt16()\
        .clip(region_i_vec)\
        .rename('hand')
    
    # Use regex to match exactly followed by the year and version
    missing_i = [
        item for item in missing 
        if re.search(rf"CERRADO_{i}_[0-9]{{4}}_v11$", item)
    ]
    
    # Extract the years using regex
    years_ij = [int(year) for year in re.findall(r"[0-9]{4}", ' '.join(missing_i))]

    for j in years_ij: 
        ##########################################################  build mosaic 
        collection = ee.ImageCollection(collectionId)\
                .filter(ee.Filter.date(str(j) + dateStart, str(j) + dateEnd))\
                .filter(ee.Filter.bounds(region_i_vec))\
                .select(spectralBands)
            
        ## apply scaling factor    
        collection = collection.map(
                lambda image: image.multiply(10000).copyProperties(image, ['system:time_start', 'system:time_end'])
            )
        
        ## apply SMA
        collection = collection.map(
                lambda image: getFractions(image, endmembers)
            )
        
        ## calculate SMA indexes        
        collection = collection\
                .map(getNDFI)\
                .map(getSEFI)\
                .map(getWEFI)\
                .map(getFNS)

        ## calculate Spectral indexes  
        collection = collection\
                .map(getCAI)\
                .map(getEVI2)\
                .map(getGCVI)\
                .map(getHallCover)\
                .map(getHallHeigth)\
                .map(getNDVI)\
                .map(getNDWI)\
                .map(getPRI)\
                .map(getSAVI)
        
        mosaic = getMosaic(
                collection= collection,
                dateStart= str(j) + dateStart,
                dateEnd=  str(j) + dateEnd,
                percentileBand= 'ndvi',
                percentileDry= 25,
                percentileWet=75)
            
            ## get other bands
        mosaic = getSlope(mosaic)
        mosaic = getEntropyG(mosaic)
        mosaic = mosaic.clip(region_i_vec)

        ## Join the mapbiomas mosaic with the auxiliary bands
        mosaic = mosaic\
            .addBands(lat)\
            .addBands(lon_sin)\
            .addBands(lon_cos)\
            .addBands(hand)\
            .addBands(fire_age.select('classification_' + str(j)).rename('fire_age').clip(region_i_vec))\
            .addBands(ee.Image(j).int16().rename('year'))
        
         ## Limit water samples only to 175 samples (avoid over-estimation)
        water_samples = ee.FeatureCollection(training_dir + 'v' + str(samples_version) + '/train_col9_reg' + str(i) + '_' + str(j) + '_v' + str(samples_version))\
            .filter(ee.Filter.eq("reference", 33))\
            .filter(ee.Filter.eq("hand", 0))\
            .limit(175)                        ## insert water samples limited to 175 
        
        ## Merge filtered water samples with other classes
        training_ij = ee.FeatureCollection(training_dir + 'v' + str(samples_version) + '/train_col9_reg' + str(i) + '_' + str(j) + '_v' + str(samples_version))\
            .filter(ee.Filter.neq("reference", 33))\
            .merge(water_samples) 
        
        bandNames_list = mosaic.bandNames().getInfo() 

        ## Train classifier
        classifier = ee.Classifier.smileRandomForest(
        numberOfTrees= 300,
        variablesPerSplit= math.floor(math.sqrt(len(bandNames_list))))\
            .train(training_ij, 'reference', bandNames_list)
        
        ## Perform classification and mask only to region 
        predicted = mosaic.classify(classifier)\
            .updateMask(region_i_ras)\
            .rename('classification_' + str(j))
        
        ## Set properties
        toExport = predicted\
            .set('collection', '9')\
            .set('version', output_version)\
            .set('biome', 'CERRADO')\
            .set('mapb', int(i))\
            .set('year', int(j))
        
        ## Create filename
        file_name = 'CERRADO_' + str(i) + '_' + str(j) + '_v' + output_version
        
        ## Build task
        task = ee.batch.Export.image.toAsset(
        image= toExport,
        description= file_name,
        assetId= output_asset + file_name,
        scale= 30,
        maxPixels= 1e13,
        pyramidingPolicy= {".default": "mode"},
        region= region_i_ras.geometry()
        )
        
        ## Export 
        task.start()
        

    print ('------------> NEXT REGION --------->')

print('All tasks have been started. Now wait a few hours and have fun :)')
  

processing region: 1
------------> NEXT REGION --------->
processing region: 2
------------> NEXT REGION --------->
processing region: 3
------------> NEXT REGION --------->
processing region: 4
------------> NEXT REGION --------->
processing region: 5
------------> NEXT REGION --------->
processing region: 6
------------> NEXT REGION --------->
processing region: 7
------------> NEXT REGION --------->
processing region: 8
------------> NEXT REGION --------->
processing region: 9
------------> NEXT REGION --------->
processing region: 10
------------> NEXT REGION --------->
processing region: 11
------------> NEXT REGION --------->
processing region: 12
------------> NEXT REGION --------->
processing region: 13
------------> NEXT REGION --------->
processing region: 14
------------> NEXT REGION --------->
processing region: 15
------------> NEXT REGION --------->
processing region: 16
------------> NEXT REGION --------->
processing region: 17
------------> NEXT REGION --------->
proces